# Lab 8, part 2: Convolutional Neural Network sample
by Domrachev Ivan, B20-Ro-01

In [9]:
from keras.datasets import mnist
from nn_from_scratch.optimizers import GradientDescent
from nn_from_scratch.examples.simple_nn import NeuralNetwork
from nn_from_scratch.nodes import ReLU, SoftMaxLoss, Vectorization
from nn_from_scratch.neurons import Linear, Convolution
from nn_from_scratch.interfaces import Neuron
import tensorflow as tf
from tensorflow.keras import layers as t_layers


import numpy as np
from matplotlib import pyplot as plt

## Part 1. One layer NN

Finally, everything is ready to create a Convolutional Neural Network!

Conviniently, the framework from the simple Neural Network is suitable for the CNN as well, so let's utilize it:

In [10]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
# CNN are slow...
train_X, train_y = train_X[:1000], train_y[:1000]
# Shuffle

p = np.random.permutation(len(train_X))
train_X, train_y = train_X[p], train_y[p]

n_input, n_output, batch_size, n_channels = 28**2, 10, 50, 1
assert train_X.shape[0] % batch_size == 0

train_X = train_X.reshape(train_X.shape[0] // batch_size, batch_size, n_channels, 28, 28)
train_y_ohe = np.zeros((len(train_y), n_output))
train_y_ohe[np.arange(len(train_y)), train_y] = 1
train_y_ohe = train_y_ohe.reshape((train_y.shape[0] // batch_size, batch_size, n_output))

The values preferably should belong to $[0; 1]$:

In [ ]:
train_X = train_X / 255
test_X = test_X / 255

Now, let's train!

In [8]:
gds = GradientDescent(lr=1e8)
layers = [
    Convolution(
        (batch_size, n_channels, 28, 28), 
        kernel_size=3, 
        output_layers=4,
        use_bias=False
    ),
    ReLU((batch_size, 4, 26, 26)),
    Convolution(
        (batch_size, 4, 26, 26), 
        kernel_size=7,
        output_layers=2,
        use_bias=False
    ),
    ReLU((batch_size, 2, 20, 20)),
    Vectorization((batch_size, 2, 20, 20)),
    Linear((batch_size, 800), (batch_size, n_output))
]
loss_fn = SoftMaxLoss((batch_size, n_output))
network = NeuralNetwork(
    n_input=n_input, 
    n_output=n_output, 
    batch_size=batch_size, 
    optimizer=gds,
    layers=layers,  
    loss_fn=loss_fn
)
network.fit(train_X, train_y_ohe, n_epochs=2)

 35%|███▌      | 7/20 [00:26<00:49,  3.81s/it, loss=2.34]


KeyboardInterrupt: 

Great, loss is decreasing very slowly. Strange, let's check the predictions:

In [5]:
def compute_accuracy(test_X: np.array, test_y: np.array, model) -> float:
    correct_predictions = 0
    total = 0

    for test_inputs, test_labels in zip(test_X, test_y):
        predicts = model.predict(test_inputs)
        pred_class = np.argmax(predicts, axis=1)

        correct_predictions += (pred_class == test_labels).sum()
        total += len(test_labels)

    return correct_predictions / total

In [6]:
test_X_batches = test_X.reshape((test_X.shape[0] // batch_size, batch_size, 28, 28))
test_y_batches = test_y.reshape((test_y.shape[0] // batch_size, batch_size,))

acc = compute_accuracy(test_X_batches, test_y_batches, network)
print(f"Accuracy: {acc}")

Accuracy: 0.1053


Well, they are random... Maybe the model itself is invalid, let's train tensorflow analogue

## Part 2. Learning TensorFlow network

This part is straight-forward, would not explain much:

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = np.expand_dims(train_X / 255, axis=-1)
test_X = np.expand_dims(test_X / 255, axis=-1)

In [ ]:
tf_layers = [
    t_layers.Conv2D(
        4, 3,
        input_shape=train_X.shape[1:],
        use_bias=False,
    ),
    t_layers.ReLU(),
    t_layers.Conv2D(
        2, 7,
        use_bias=False,
    ),
    t_layers.ReLU(),
    t_layers.Flatten(),
    t_layers.Dense(
        10,
        use_bias=True,
    )
]
model = tf.keras.models.Sequential()
for l in tf_layers:
    model.add(l)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 4)         36        
                                                                 
 re_lu (ReLU)                (None, 26, 26, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 2)         392       
                                                                 
 re_lu_1 (ReLU)              (None, 20, 20, 2)         0         
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 10)                8010      
                                                                 
Total params: 8438 (32.96 KB)
Trainable params: 8438 (32

2023-11-13 21:15:36.409917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-13 21:15:36.410788: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_X, train_y, epochs=5, 
                    validation_data=(test_X, test_y))

Epoch 1/5


2023-11-13 21:15:36.700702: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1460/1875 [======================>.......] - ETA: 4s - loss: 0.6234 - accuracy: 0.7996

KeyboardInterrupt: 

The accuracy there is not random, and the network performs great. Maybe the weights are different?..

## Part 3. Comparison with TensorFlow

We should compare:
1. Outputs of the model (given the same initial weights and the same input)
2. Partial derivative of loss w.r.t. input. If they are close enough, then all the inner states are similar as well:

In [ ]:
example_pic, example_label = train_X[0], train_y_ohe[0]

In [ ]:
state = example_pic.copy()
for layer in layers:
    state = layer.forward(state)

loss = loss_fn.forward(state, example_label)
dL_dy = loss_fn.backward()
partial_derivative = dL_dy
dL_dx = []
dL_dw = []

for layer in layers[::-1]:
    partial_derivative = layer.backward(partial_derivative)
    dL_dx.append(partial_derivative)

    if isinstance(layer, Neuron):
        dL_dw.append(layer._W_pd)

In [ ]:
example_pic.shape

(50, 1, 28, 28)

In [ ]:
example_pic_tf = tf.constant(
    np.moveaxis(
        example_pic,
        1, -1
    ), 
    dtype=tf.float32
)

weights_tf = [
    tf.constant(
        np.moveaxis(
            l.W,
            0, -1
        ).astype(np.float32)
    )
    for l in [layers[0], layers[2]]
]
weights_tf.extend([
    tf.constant(
        np.moveaxis(
            layers[5].W[1:, :],
            0, -1
        ).astype(np.float32)
    ),
    tf.constant(
        np.moveaxis(
            layers[5].W[0, :],
            0, -1
        ).astype(np.float32)
    ),  
])

2023-11-07 14:16:17.564457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-07 14:16:17.565690: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
example_pic_tf.shape

TensorShape([50, 28, 28, 1])

In [ ]:
tf_layers = [
    t_layers.Conv2D(
        4, 3,
        input_shape=example_pic_tf.shape[1:],
        use_bias=False,
        kernel_initializer=tf.keras.initializers.Constant(weights_tf[0])
    ),
    t_layers.ReLU(),
    t_layers.Conv2D(
        2, 7,
        use_bias=False,
        kernel_initializer=tf.keras.initializers.Constant(weights_tf[1])
    ),
    t_layers.ReLU(),
    t_layers.Flatten(),
    t_layers.Dense(
        10,
        use_bias=True,
        kernel_initializer=tf.keras.initializers.Constant(weights_tf[2]),
        bias_initializer=tf.keras.initializers.Constant(weights_tf[3])
    )
]
model = tf.keras.models.Sequential()
for l in tf_layers:
    model.add(l)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 4)         36        
                                                                 
 re_lu (ReLU)                (None, 26, 26, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 2)         392       
                                                                 
 re_lu_1 (ReLU)              (None, 20, 20, 2)         0         
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 10)                8010      
                                                                 
Total params: 8438 (32.96 KB)
Trainable params: 8438 (32

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(example_pic_tf)
    conv_output = model(example_pic_tf)

conv_output_np = conv_output.numpy()

1. The outputs are similar:

In [ ]:
np.max((conv_output_np - state[0]) < 2e-1)

True

In [ ]:
dL_dy_keras = tf.constant(np.moveaxis(dL_dy, 1, -1), dtype=tf.float32)

dL_dx_keras = tape.gradient(conv_output, example_pic_tf, output_gradients=dL_dy_keras)

dL_dw_keras = [
    tape.gradient(conv_output, layer_i.trainable_variables)
    for layer_i in [tf_layers[0], tf_layers[2], tf_layers[5]]
]

dL_dx_keras_np = np.moveaxis(dL_dx_keras.numpy(), -1, 1)
dL_dw_keras_np = [dL_dw_keras_i[0].numpy() for dL_dw_keras_i in dL_dw_keras][::-1]

In [ ]:
dL_dw[0] = dL_dw[0][1:, :]
dL_dw[1] = dL_dw[1].T
dL_dw[2] = dL_dw[2].T

In [ ]:
for a in dL_dw_keras_np:
    print(a.shape)

(800, 10)
(7, 7, 4, 2)
(3, 3, 1, 4)


2. The output of the backpropogation is similar (hence, all the inner states of back propogation are similar as well)

In [ ]:
np.all((dL_dx_keras_np - dL_dx[-1]) < 1e-1)

True

I have several ideas why results diverge and will try to fix them in future works.